In [1]:
import copy
import pandas as pd
import random
import re
import sys

inputFile = pd.read_csv(u"Управление анкетой 2.csv", header=-1, sep='\t')
inputFile = inputFile.fillna('')
lengthOfInputFile = inputFile.shape[0]
widthOfInputFile = inputFile.shape[1]
N = 1000    # Количество опрашиваемых

In [2]:
inputFile

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,#1,,,,,,,,,,,,,,
1,Из какой страны вы приехали на работу в Россию?,,,,,,,,,,,,,,
2,Узбекистан,Армения,XXX,,,,,,,,,,,,
3,0.6,0.3,0.1,,,,,,,,,,,,
4,#2,,,,,,,,,,,,,,
5,Кто Вы по национальности?,,,,,,,,,,,,,,
6,Узбек,Армянин,Киргиз,Азербайджанец,Таджик,Молдованин,Русский,Грузин,Другое (напишите),,,,,,
7,0.6,0.3,0.1,0,0,0,0,0,0,,,,,,
8,#3,,,,,,,,,,,,,,
9,Как долго Вы проживаете в России?,,,,,,,,,,,,,,


In [3]:
inputFile[0][3]

'0.6'

# Вспомогательные функции

## Добавление распределения

In [4]:
def addDistribution(distributionsDict, inputFile, iVariants, iValues, title, key):
    j = 0
    distributionsDict[title][key] = {}
    while True:
        #print 'строка ' + str(iValues) + '\tстолбец ' + str(j)
        if j >= inputFile.shape[1]:
            print 'Край таблицы'
            break
        if inputFile[j][iVariants] == '':
            print 'Кончились варианты'
            break
        if inputFile[j][iValues] == '':
            print 'Отсутствует вероятность для ' + inputFile[j][iVariants] + ' в строке ' + str(iValues)
            return -1
        distributionsDict[title][key][inputFile[j][iVariants]] = inputFile[j][iValues]
        j += 1
    
    # Проверка
    mySum = 0
    for item in distributionsDict[title][key]:
        mySum += float(distributionsDict[title][key][item])
    if mySum > 1:
        print 'Неправильно указаны вероятности для вопроса ' + str(title) + ' в строке ' + str(iValues)
        return -1
    
    print pd.DataFrame(distributionsDict[title])
    return 0

## Считывание правил по одному вопросу

In [5]:
def readQuestion():
    j = 0
    EndOfQuestion = False
    
    while not EndOfQuestion:
        errorCode = 0

        if i + j >= lengthOfInputFile:    # Обработка конца таблицы
            EndOfQuestion = True
            continue

        firstCell = inputFile[0][i + j]    # Новый вопрос знаменуется строкой с решёткой в начале
        if firstCell.lstrip('#') == str(questionNumber + 1):
            EndOfQuestion = True
            continue
            
        
        
        switch[TypeOfQuestion]
        

        j += 1

### Вопрос с одним вариантом ответа

In [6]:
def singleChoiceQuestion():
    if j == 2:    # Распределение по умолчанию
        errorCode = addDistribution(distributionsDict, inputFile, i + j, i + j + 1, 'q' + str(questionNumber), 'Default')

    if firstCell.find('&&&') != -1: # Распределение с логическим условием
        k = 1
        key = ['&&&']
        while inputFile[k][i + j] != '':
            key.append( (inputFile[k][i + j], inputFile[k + 1][i + j]) ) 
            if k+1 == widthOfInputFile:
                break
            k += 2
        key = tuple(key)

        errorCode = addDistribution(distributionsDict, inputFile, i + 2, i + j + 1, 'q' + str(questionNumber), key)

    if ( errorCode == -1 ):
        print 'Поправьте распределение!'
        sys.exit()

## Добавление ответа

In [7]:
def addAnswer(answersDict, title, distribution):
    answer = random.random()
    #print answer
    i = 1
    mySum = 0
    for item in distribution:
        #print item
        #print distribution[item]
        mySum += float(distribution[item])
        #print mySum
        if answer < mySum:
            #print 'answer ' + str(answer) + '\t<\tmySum ' + str(mySum)
            answer = i
            #print 'Final answer:\t' + str(answer)
            break
        #else:
            #print 'answer ' + str(answer) + '\t>\tmySum ' + str(mySum)
        i += 1
    #print answer
    answersDict[title].append(answer)
    return 0

# Основное тело программы
## Создаём ключи словаря

In [8]:
questionsDict = {}
for i in range(1, 23):
    questionsDict['q' + str(i)] = {}

## Заполняем правила

In [9]:
switch = {
    'SingleChoice': singleChoiceQuestion(),
    'MultipleChoice': multipleChoiceQuestion()
}
distributionsDict = copy.deepcopy(questionsDict)
questionNumber = 0

for i in range(lengthOfInputFile):  # Идём по каждой строке исходной таблицы
    
    firstCell = inputFile[0][i]
    if firstCell.lstrip('#') == str(questionNumber + 1):    # Нашли что-то похожее на вопрос?
        questionNumber += 1
    else:
        continue
    
    # Заменить на лямбда?
    TypeOfQuestion = 'SingleChoice'
    if inputFile[1][i]: # Точно не будет всегда истиной?
        TypeOfQuestion = inputFile[1][i]
    
    switch[TypeOfQuestion]
    
    #questionsDict['q' + str(questionNumber)]

NameError: global name 'j' is not defined

# Формируем заголовки таблицы ответов

In [ ]:
answersDict = copy.deepcopy(questionsDict)
for i in range(1, 23):
    title = 'q' + str(i)
    answersDict[title] = []

# Бегаем по опрашиваемым

In [ ]:
for i in range(N):
    questionNumber = 0
    for j in range(1, 23):
        
        title = 'q' + str(j)
        
        
        if j == 21:    #заглушка
            answersDict[title].append(0)
            continue
            
        key = 'Default'
        for distribution in distributionsDict[title]:
            if distribution[0] == '&&&':
                if answersDict['q'+str(distribution[1][0])] == str(distribution[1][1]):
                    key = distribution
        
        addAnswer(answersDict, title, distributionsDict[title][key])

# Экспорт результатов

In [ ]:
result = pd.DataFrame(answersDict)

In [ ]:
result = result.reindex(sorted(sorted(result.columns), key = lambda myStr: len(myStr)), axis=1)

In [ ]:
result.to_csv('output.csv', sep='\t', header=True, index=False)